In [ ]:
from datetime import datetime, timedelta
import csv
import sqlite3
import requests
from bs4 import BeautifulSoup
import json
from typing import Dict, List
import re

In [ ]:
#===================== SCRAP SAMPLE PRODUCTS FOR DB =======================

BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"

def scrape_books(pages=25):
    books = []

    for page in range(1, pages + 1):
        url = BASE_URL.format(page)
        print(f"Scraping page {page}...")

        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all("article", class_="product_pod")

        for article in articles:
            name = article.h3.a["title"]

            price_text = article.find("p", class_="price_color").text
            price = float(re.sub(r"[^\d.]", "", price_text))


            books.append({
                "name": name,
                "price": price,
                #"source": f"Page {page}"
            })

    return books

In [ ]:
# ============== Scrap all the pages ====================

books = scrape_books(25)

print(f"\nTotal books scraped: {len(books)}")

for book in books[:2]:
    print(book)

In [ ]:
# =============== Place into a CSV file ================

filename = "inventory_sample.csv"

with open(filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=books[0].keys())
    writer.writeheader()
    writer.writerows(books)

print("Data saved to inventory_sample.csv")

In [ ]:
# ================ DATABASE SETUP =====================

# Create or connect to the database
conn = sqlite3.connect("Inventory_mgnt.db")
conn.execute("PRAGMA foreign_keys = ON")  # Enforces foreign key constraints - execute function id=s needed to execute queries...
cursor = conn.cursor()  # this executes the query

In [ ]:
# ================== Create a Product table ====================

cursor.execute("""     
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price FLOAT NOT NULL,
    supplier VARCHAR DEFAULT 'Generic'
               
)
""")
conn.commit() #saves any changes made to the database permanently. 

In [ ]:
# ============== Create a Inventory table ==============

cursor.execute("""     
CREATE TABLE IF NOT EXISTS inventory (
    inventory_id    INTEGER PRIMARY KEY AUTOINCREMENT,           
    product_id      INTEGER NOT NULL,
    name            TEXT NOT NULL,
    stock           INTEGER NOT NULL CHECK (stock >= 0) DEFAULT 20,
    reorder_level   INTEGER NOT NULL CHECK (reorder_level >= 5) DEFAULT 5,
    units_sold      INTEGER NOT NULL DEFAULT 0,
    FOREIGN KEY (product_id) REFERENCES product(product_id) ON DELETE CASCADE
)
""")
conn.commit()

In [ ]:
# =============Read CSV and Insert Into Tables ==================

def insert_from_csv(csv_file):
    conn = sqlite3.connect("Inventory_mgnt.db")
    cursor = conn.cursor()
    
    with open(csv_file, newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)

        for row in reader:
            name = row["name"]
            price = float(row["price"])
            
            # Insert into product
            cursor.execute("""
                INSERT INTO product (name, price)
                VALUES (?, ?)
            """, (name, price))

            product_id = cursor.lastrowid

            # Insert into inventory
            #cursor.execute("""
            #    INSERT INTO inventory (product_id, stock, reorder_level, units_sold)
            #    VALUES (?, ?, ?, 0)
            #""", (product_id, stock, reorder_level))

    conn.commit()
    conn.close()

insert_from_csv("inventory_sample.csv")
print("Data inserted into table successfully")

In [ ]:
# =======================CLASS SETUP =======================

class Product:
    def __init__(self, name, price, supplier): 
        self.product_id = None
        self.name = name
        self.price = price
        self.supplier = supplier
        self.stock = 20
        self.reorder_level = 5
        self.units_sold = 0

    def sell(self, quantity):
        if quantity > self.stock:
            print(f"Not enough stock for {self.name}")
            return False

        self.stock -= quantity
        self.units_sold += quantity
        return True

    def needs_restock(self):
        return self.stock <= self.reorder_level

    def __str__(self):
        return f"{self.name} | Stock: {self.stock} | Sold: {self.units_sold}"
    
    
    
    


In [ ]:
class Inventory:
    def __init__(self, db_name = "inventory_mgnt.db"):
        self.products = {}
        self.db_name = db_name

    #ADD PRODUCTS TO DB
    #ADMIN ACCESS: A PASSWORD IS NEEDED TO PROCEED
    def add_product(self, product, stock, reorder_level=5):
        password = input('ENTER PASSWORD') #ENTER PASSWORD TO PROCEED
        if password == 'admin':

            conn = sqlite3.connect("Inventory_mgnt.db")
            cursor = conn.cursor()    

        # Insert into product table
            cursor.execute("""
                INSERT INTO product (name, price, supplier)
                VALUES (?, ?, ?)
            """, (product.name, product.price, product.supplier))

        # Get generated primary key
            product.product_id = cursor.lastrowid
            product.stock = 0
            product.supplier = 'generic'
            product.reorder_level = 5


            # Insert into inventory table
            cursor.execute("""
            INSERT INTO inventory (product_id, stock, reorder_level, units_sold)
            VALUES (?, ?, ?, 0)
            """, (product.product_id, stock, reorder_level))
        
            # Store in memory
            self.products[product.product_id] = product

            print(f"Added {product.name} (ID: {product.product_id})")
        
        
            conn.commit()
            conn.close()

        else:
            print("You dont have admin priviledge")    

     #--------------------------------------------------------  
     # SELL PRODUCT
     # IT ALERTS WHEN STOCK IS LOW
    def sell_product(self, product_id, quantity):
            conn = sqlite3.connect("Inventory_mgnt.db")
            cursor = conn.cursor()  

            # Get current stock & reorder level
            cursor.execute("""
                SELECT p.name, i.stock, i.units_sold, i.reorder_level
                FROM inventory i
                JOIN product p ON i.product_id = p.product_id
                WHERE p.product_id = ?
            """, (product_id,))

            result = cursor.fetchone()

            if not result:
                print("Product not found")
                return

            name, stock, units_sold, reorder_level = result

            # Prevent overselling
            if quantity > stock:
                print(f"Not enough stock for {name}")
                return

            # Update inventory
            cursor.execute("""
                UPDATE inventory
                SET stock = stock - ?,
                    units_sold = units_sold + ?
                WHERE product_id = ?
            """, (quantity, quantity, product_id))

            conn.commit()

            print(f"Sold {quantity} units of {name}")

            # Low-stock alert
            if stock - quantity <= reorder_level:
                print(f"Low stock alert for {name}")
    

    def low_stock_alerts(self):
        print("\n LOW STOCK ALERTS")
        for product in self.products.values():
            if product.needs_restock():
                print(f"{product.name} (Stock: {product.stock})")
    
    # ================== DELETE PRODUCTS =======================
    # ADMIN ACCESS NEEDED
    def delete_product(self, product_id):
        password = input('ENTER PASSWORD') #ENTER PASSWORD TO PROCEED
        if password == 'admin':
                                 
            conn = sqlite3.connect("Inventory_mgnt.db")
            cursor = conn.cursor()    
            
            cursor.execute("PRAGMA foreign_keys = ON")

            # Delete from product table
            cursor.execute("""
                DELETE FROM product
                WHERE PRODUCT_ID = ?
            """, (product_id,))

                        
            if cursor.rowcount == 0:
                print("Product not found")
                return

            print(f"Product {product_id} deleted successfully")
        
        
            conn.commit()
            conn.close()

        else:
            print("You dont have admin priviledge") 

    # CHECK IF RESTOCK IS NEEDED
    def needs_restock(self, product_id):
        conn = sqlite3.connect("Inventory_mgnt.db")
        cursor = conn.cursor()   

        cursor.execute("""
                SELECT p.name, i.stock,i.reorder_level
                FROM inventory i
                JOIN product p ON i.product_id = p.product_id
                WHERE p.product_id = ?
            """, (product_id,))

        result = cursor.fetchone()

        if not result:
                print("Product not found")
                return False

        name, stock, reorder_level = result

        if stock <= reorder_level:
                print(f"{name} needs restocking Stock: {stock}, Reorder level: {reorder_level})")
                return True

        print (f"{name} doesn't need restocking")
        #return False
    
        conn.commit()
        conn.close()

    
    def top_selling_products(self, limit=3):
        conn = sqlite3.connect("Inventory_mgnt.db")
        cursor = conn.cursor() 

        cursor.execute("""
            SELECT p.name, i.units_sold
            FROM inventory i
            JOIN product p ON i.product_id = p.product_id
            ORDER BY i.units_sold DESC
            LIMIT ?
        """, (limit,))

        results = cursor.fetchall()

        if not results:
            print("No sales data found")
            return []

        print(f"Top {limit} best-selling products:")
        for name, sold in results:
            print(f"- {name}: {sold} units sold")

        return results
    


    def worst_selling_products(self, limit=3):
        conn = sqlite3.connect("Inventory_mgnt.db")
        cursor = conn.cursor() 

        cursor.execute("""
            SELECT p.name, i.units_sold
            FROM inventory i
            JOIN product p ON i.product_id = p.product_id
            ORDER BY i.units_sold ASC
            LIMIT ?
        """, (limit,))

        results = cursor.fetchall()

        if not results:
            print("No sales data found")
            return []

        print(f"Top {limit} best-selling products:")
        for name, sold in results:
            print(f"- {name}: {sold} units sold")

        return results
    

    def restock_products_recommendation(self, limit=3):
        conn = sqlite3.connect("Inventory_mgnt.db")
        cursor = conn.cursor() 

        cursor.execute("""
            SELECT p.name, i.units_sold, i.stock, i.reorder_level
            FROM inventory i
            JOIN product p ON i.product_id = p.product_id
            WHERE i.stock <= i.reorder_level
            LIMIT ?
        """, (limit,))

        results = cursor.fetchall()

        if not results:
            print("No sales data found")
            return []

        
        for name, units_sold, stock, reorder_level in results:
            recommended_amount = units_sold * 2
            print(f"Book Name: {name} \nRecommended amount to restock is : {recommended_amount}\n")

        return #results

In [ ]:
# ADD PRODUCTS

inventory = Inventory()

product = Product("City of Baltimore", 18.99, "Mc Hill")
inventory.add_product(product, stock=10, reorder_level=5)

In [ ]:
# SELL PRODUCTS

inventory = Inventory()
inventory.sell_product(4, 5)

In [ ]:
#DELETE PRODUCT

inventory = Inventory()
inventory.delete_product(2)

In [ ]:
#NEEDS RESTOCK LIST

inventory = Inventory()
inventory.needs_restock(1)

In [ ]:
#TOP SELLING PRODUCTS

inventory = Inventory()
inventory.top_selling_products(3)

In [ ]:
#WORST SELLLING PRODUCTS

inventory = Inventory()
inventory.worst_selling_products(3)

In [ ]:
#Recommend how many products store owner should buy

inventory = Inventory()
inventory.restock_products_recommendation()